# 스케일링
- 회귀분석에서의 스케일링의 목적과 방법

## 1. 조건수
- `조건수 conditional number` : 가장 큰 고유치(고윳값)와 가장 작은 고유치(고윳값)의 비율
- 회귀분석에서는 공분산행렬 X^TX의 가장 큰 고유치와 가장 작은 고유치의 비율
    - 선형회귀모형에서 RSS와, 그레디언트 벡터, 헤시안행렬까지 공분산행렬 X^TX가 나온다. 
    - $\text{conditional number}=\dfrac{\lambda_{max}}{\lambda_{min}}, \;\; (\lambda = \text{고유값})$
    - min 고윳값과 max 고윳값이 어느정도 차이나는지에 대한 비율
- 조건수 계산 명령어
    - **np.linalg.cond(A)**
- 확률분포 편의 **다변수 정규분포와 고유값 분해** 파트에서 조건수의 원리를 다룬다.    

### 1-1. 조건수의 의미
- 연립방정식 Ax=b 에서 계수행렬인 A의 조건수가 크면, 즉 A의 min과 max의 비율이 크면, 계수행렬 A와 상수벡터 b에 대한 해 x의 민감도가 커진다.
    - 계수행렬 A의 조건수가 크면 -> x의 민감도가 커진다.
- 민감도가 커지면 계수행렬과 상수벡터에 존재하는 오차가 해 x에 미치는 영향이 크다는 것을 의미한다.
- 조건수가 가장 작은 경우는 행렬 A가 단위 행렬인 경우이다. 
    - max와 min 값이 같은 경우
- **공분산행렬 X^TX의 조건수가 크면 회귀분석을 사용한 예측값도 오차가 커지게 된다.** 
    - 선형회귀모형의 해인 w 계수벡터를 구하는 OLS 공식
    - $w^{*} = (X^TX)^TX^Ty$

### 1-2. 조건수의 예시
- 계수행렬이나 상수벡터에 약간의 오차가 있는 경우 해의 변화
- 조건수가 높다면 해의 민감도가 높고, 해의 민감도가 높다는 것은 계수행렬이나 상수벡터의 오차로 인한 해의 결과가 크게 바뀔 수 있다는 것을 의미한다.

#### 조건수가 낮은 경우
- cond(A) = 1/1 = 1

In [25]:
A = np.eye(4)
A

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [26]:
b = np.ones(4)
x = sp.linalg.solve(A, b)
x

array([1., 1., 1., 1.])

#### 해 x는 계수행렬의 역행렬을 상수벡터와 곱하여 구할 수 있다.

In [27]:
sp.linalg.inv(A) @ b

array([1., 1., 1., 1.])

#### 계수행렬에 오차가 있는 경우
- 계수행렬의 조건수가 1로 매우 작으므로 해에 별다른 영향을 끼치지 않는다.

In [31]:
x_error = sp.linalg.solve(A + 0.0001 * np.eye(4), b)
x_error

array([0.99990001, 0.99990001, 0.99990001, 0.99990001])

In [30]:
A + (0.0001 * np.eye(4))

array([[1.0001, 0.    , 0.    , 0.    ],
       [0.    , 1.0001, 0.    , 0.    ],
       [0.    , 0.    , 1.0001, 0.    ],
       [0.    , 0.    , 0.    , 1.0001]])

#### 조건수가 큰 행렬의 경우
- 조건수가 큰 힐버트 행렬을 계수행렬로 오차의 영향이 어떻게 달라지는 지 확인

In [32]:
A = sp.linalg.hilbert(4)
A

array([[1.        , 0.5       , 0.33333333, 0.25      ],
       [0.5       , 0.33333333, 0.25      , 0.2       ],
       [0.33333333, 0.25      , 0.2       , 0.16666667],
       [0.25      , 0.2       , 0.16666667, 0.14285714]])

In [33]:
np.linalg.cond(A)

15513.73873892924

In [34]:
sp.linalg.solve(A, b)

array([  -4.,   60., -180.,  140.])

#### 조건수가 큰 계수행렬에 오차가 생기면?
- A의 조건수가 매우 크므로 아주 작은 오차가 발생한 경우 해의 값이 크게 달라진다.

In [35]:
sp.linalg.solve(A + 0.0001 * np.eye(4), b)

array([ -0.58897672,  21.1225671 , -85.75912499,  78.45650825])

## 2. 회귀분석과 조건수
- 회귀분석에서 조건수가 커지는 경우 2가지
- 1) 변수들의 단위 차이로 인해 숫자의 스케일이 크게 달라지는 경우
    - 이러한 경우는 **스케일링 (scaling)** 을 사용한다.
- 2) 다중 공선성 (multicolinearity), 즉 상관관계가 큰 독립 변수들이 있는 경우. 
    - 이러한 경우는 **변수 선택**이나 **PCA**를 사용한 **차원축소** 방법으로 해결한다.

### 2-1. 독립변수의 분포 모양으로 조건수 확인
- 다변수정규분포 함수를 고유분해하면, 공분산행렬의 고윳값이 타원의 폭(min)이 되고, 공분산 행렬의 고유벡터가 타원의 방향(max)이 된다.
    - 스케일링이 안된 경우
    - 다중공선성이 있는 경우(독립변수간에 상관관계로 인한)

## 3. 보스턴 집값 데이터에서 조건수 확인
- OLS summary 레포트의 제일 하단에 조건수에 대한 메시지

In [10]:
from sklearn.datasets import load_boston

boston = load_boston()

dfX0 = pd.DataFrame(boston.data, columns=boston.feature_names)
dfx = sm.add_constant(dfX0)
dfy = pd.DataFrame(boston.target, columns=["MEDV"])
df = pd.concat([dfy, dfx], axis=1)

In [11]:
dfx.head()

,const,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,1.0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,1.0,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,1.0,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,1.0,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,1.0,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [12]:
dfy.head()

,MEDV
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2


In [13]:
df.head()

,MEDV,const,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,24.0,1.0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,21.6,1.0,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,34.7,1.0,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,33.4,1.0,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,36.2,1.0,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


#### OLS 클래스의 fomula 사용시 독립변수들 입력 방법
- "+".join(컬럼명)

In [14]:
model = sm.OLS.from_formula("MEDV ~ " + "+".join(boston.feature_names), data=df)
model

In [6]:
a = "y~"
a

'y~'

In [8]:
a + "+".join(boston.feature_names)

'y~CRIM+ZN+INDUS+CHAS+NOX+RM+AGE+DIS+RAD+TAX+PTRATIO+B+LSTAT'

In [16]:
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Sat, 18 Jun 2022   Prob (F-statistic):          6.72e-135
Time:                        15:11:42   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     36.4595      5.103      7.144      0.0

### 3-1. 독립변수들의 표준편차 
- 독립변수들의 데이터의 표준편차가 서로 다르다.
- 독립변수가 스케일링이 안되면 조건수가 커져서 예측 오차가 증폭될 가능성이 커진다.

In [39]:
dfx.describe().loc["std"][1:]

CRIM         8.601545
ZN          23.322453
INDUS        6.860353
CHAS         0.253994
NOX          0.115878
RM           0.702617
AGE         28.148861
DIS          2.105710
RAD          8.707259
TAX        168.537116
PTRATIO      2.164946
B           91.294864
LSTAT        7.141062
Name: std, dtype: float64

### 3-2. 특정 독립변수의 크기를 늘려서 조건수 확인
- dfx2의 TAX 독립변수에 임의의 수를 곱해준다. 
- 종속변수와 합하여 다시 새로운 데이터 프레임을 만든다.
- 조건수가 엄청나게 큰 수로 증가함.
- 모델의 성능을 의미하는 R-squared 값이 감소함

In [41]:
dfx2 = dfx.copy()
dfx2["TAX"] *= 1e13
df2 = pd.concat([dfx2, dfy], axis=1)

In [43]:
model2 = sm.OLS.from_formula("MEDV~" + "+".join(boston.feature_names), data=df2)
result2 = model2.fit()

print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.333
Model:                            OLS   Adj. R-squared:                  0.329
Method:                 Least Squares   F-statistic:                     83.42
Date:                Sat, 18 Jun 2022   Prob (F-statistic):           8.42e-44
Time:                        19:02:20   Log-Likelihood:                -1737.9
No. Observations:                 506   AIC:                             3484.
Df Residuals:                     502   BIC:                             3501.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0038      0.000     -8.554      0.0

In [54]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Sat, 18 Jun 2022   Prob (F-statistic):          6.72e-135
Time:                        19:06:58   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     36.4595      5.103      7.144      0.0

### 3-3. 스케일링
- statsmodels의 모형지정 문자열에서 scale(컬럼명) 명령으로 스케일링 가능하다.
    - 스케일링 값은 따로 저장된다. predict() 명령을 사용할 때 같은 스케일을 사용하면 된다.
- 카테고리 더미 변수인 CHAS는 스케일을 하지 않는다.
- 조건수가 10.6으로 줄어들었다.

In [55]:
feature_names = list(boston.feature_names)
feature_names

['CRIM',
 'ZN',
 'INDUS',
 'CHAS',
 'NOX',
 'RM',
 'AGE',
 'DIS',
 'RAD',
 'TAX',
 'PTRATIO',
 'B',
 'LSTAT']

In [56]:
feature_names.remove("CHAS")
feature_names

['CRIM',
 'ZN',
 'INDUS',
 'NOX',
 'RM',
 'AGE',
 'DIS',
 'RAD',
 'TAX',
 'PTRATIO',
 'B',
 'LSTAT']

#### 독립변수의 이름을 scale(컬럼명) 형태로 만들어 준다.
- OLS 모델에서 스케일링을 하기 위해서

In [57]:
feature_names = ["scale({})".format(name) for name in feature_names] + ["CHAS"]
feature_names

['scale(CRIM)',
 'scale(ZN)',
 'scale(INDUS)',
 'scale(NOX)',
 'scale(RM)',
 'scale(AGE)',
 'scale(DIS)',
 'scale(RAD)',
 'scale(TAX)',
 'scale(PTRATIO)',
 'scale(B)',
 'scale(LSTAT)',
 'CHAS']

In [59]:
"MEDV ~ " + "+".join(feature_names)

'MEDV ~ scale(CRIM)+scale(ZN)+scale(INDUS)+scale(NOX)+scale(RM)+scale(AGE)+scale(DIS)+scale(RAD)+scale(TAX)+scale(PTRATIO)+scale(B)+scale(LSTAT)+CHAS'

In [58]:
model3 = sm.OLS.from_formula("MEDV~" + "+".join(feature_names), data=df)
result3 = model3.fit()

print(result3.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Sat, 18 Jun 2022   Prob (F-statistic):          6.72e-135
Time:                        19:13:18   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         22.3470      0.219    101.